## Hexapod Forward Kinematics with Newton Method + Regression First Guess ##

In [5]:
import numpy as np
from numpy import cos, sin, pi, exp
from math import fabs, sqrt
import time
import random
from IPython.display import clear_output

In [6]:
def Hexapod_Constructive_Vectors():

    # Hexapod Constructive Properties ========================================================================
    Rground = 79.91    # Ground Base of the Hexapod Radius (mm)
    Rbase = 49.659     # Mobile Platform of the Hexapod Radius (mm)

    angleground = 23.47    # Ground Base - Closest Angle between the joints
    anglebase = 50.03      # Mobile Platform - Closest Angle between the joints

    # Calculation of the joints Angles for the definition of the base and platform vectors
    a_theta1 = 90 - (angleground/2)
    a_theta2 = a_theta1 + angleground
    a_theta3 = a_theta2 + (360 - 3*angleground)/3
    a_theta4 = a_theta3 + angleground
    a_theta5 = a_theta4 + (360 - 3*angleground)/3
    a_theta6 = a_theta5 + angleground

    b_theta1 = 90 - (anglebase/2)
    b_theta2 = b_theta1 + anglebase
    b_theta3 = b_theta2 + (360 - 3*anglebase)/3
    b_theta4 = b_theta3 + anglebase
    b_theta5 = b_theta4 + (360 - 3*anglebase)/3
    b_theta6 = b_theta5 + anglebase

    # Hexapod Position in the Space = [0, 0, 0]

    J = 0

    a_theta1 = np.radians(a_theta1+J)
    a_theta2 = np.radians(a_theta2+J)
    a_theta3 = np.radians(a_theta3+J)
    a_theta4 = np.radians(a_theta4+J)
    a_theta5 = np.radians(a_theta5+J)
    a_theta6 = np.radians(a_theta6+J)

    b_theta1 = np.radians(b_theta1+J)
    b_theta2 = np.radians(b_theta2+J)
    b_theta3 = np.radians(b_theta3+J)
    b_theta4 = np.radians(b_theta4+J)
    b_theta5 = np.radians(b_theta5+J)
    b_theta6 = np.radians(b_theta6+J)

    # Vector a1...a6 - Location of the ground joints in the reference coordinate system:
    a1 = np.array((Rground*np.cos(a_theta1), Rground*np.sin(a_theta1), 0)) + (0, 0, 0)
    a2 = np.array((Rground*np.cos(a_theta2), Rground*np.sin(a_theta2), 0)) + (0, 0, 0)
    a3 = np.array((Rground*np.cos(a_theta3), Rground*np.sin(a_theta3), 0)) + (0, 0, 0)
    a4 = np.array((Rground*np.cos(a_theta4), Rground*np.sin(a_theta4), 0)) + (0, 0, 0)
    a5 = np.array((Rground*np.cos(a_theta5), Rground*np.sin(a_theta5), 0)) + (0, 0, 0)
    a6 = np.array((Rground*np.cos(a_theta6), Rground*np.sin(a_theta6), 0)) + (0, 0, 0)

    a = (a1, a2, a3, a4, a5, a6)

    # Vector b1...b6 - Location of the platform joints in the platform coordinate system:
    b1 = np.array((Rbase*np.cos(b_theta1), Rbase*np.sin(b_theta1), 0))
    b2 = np.array((Rbase*np.cos(b_theta2), Rbase*np.sin(b_theta2), 0))
    b3 = np.array((Rbase*np.cos(b_theta3), Rbase*np.sin(b_theta3), 0))
    b4 = np.array((Rbase*np.cos(b_theta4), Rbase*np.sin(b_theta4), 0))
    b5 = np.array((Rbase*np.cos(b_theta5), Rbase*np.sin(b_theta5), 0))
    b6 = np.array((Rbase*np.cos(b_theta6), Rbase*np.sin(b_theta6), 0))

    b = (b1, b2, b3, b4, b5, b6)
    
    return a, b

In [7]:
def Rotation_XYZ(thetaRX, thetaRY, thetaRZ):

    thetaRX = np.radians(thetaRX)
    thetaRY = np.radians(thetaRY)
    thetaRZ = np.radians(thetaRZ)

    RX = np.array([[1, 0, 0],
          [0, np.cos(thetaRX), np.sin(thetaRX)],
          [0, -np.sin(thetaRX), np.cos(thetaRX)]])

    RY = np.array([[np.cos(thetaRY), 0, -np.sin(thetaRY)],
          [0, 1, 0],
          [np.sin(thetaRY), 0, np.cos(thetaRY)]])

    RZ = np.array([[np.cos(thetaRZ), np.sin(thetaRZ), 0],
          [-np.sin(thetaRZ), np.cos(thetaRZ), 0],
          [0, 0, 1]])

    # Transposition in order to keep the rotations related to the reference:
    RX = np.transpose(RX)
    RY = np.transpose(RY)
    RZ = np.transpose(RZ)

    R = np.matmul(RZ, RY)
    R = np.matmul(R, RX)

    return R

In [15]:
def Hexapod_System(x, L):
    
    a, b = Hexapod_Constructive_Vectors()
    
    P = ((x[0], x[1], x[2]))
    thetaRX, thetaRY, thetaRZ = (x[3], x[4], x[5])
    
    # Hexapod Forward Kinematics System of non-linear equations:
    eq1 = np.linalg.norm(P + Rotation_XYZ(thetaRX, thetaRY, thetaRZ).dot(b[0]) - a[0]) - L[0]
    eq2 = np.linalg.norm(P + Rotation_XYZ(thetaRX, thetaRY, thetaRZ).dot(b[1]) - a[1]) - L[1]
    eq3 = np.linalg.norm(P + Rotation_XYZ(thetaRX, thetaRY, thetaRZ).dot(b[2]) - a[2]) - L[2]
    eq4 = np.linalg.norm(P + Rotation_XYZ(thetaRX, thetaRY, thetaRZ).dot(b[3]) - a[3]) - L[3]
    eq5 = np.linalg.norm(P + Rotation_XYZ(thetaRX, thetaRY, thetaRZ).dot(b[4]) - a[4]) - L[4]
    eq6 = np.linalg.norm(P + Rotation_XYZ(thetaRX, thetaRY, thetaRZ).dot(b[5]) - a[5]) - L[5]
    
    return np.array([eq1, eq2, eq3, eq4, eq5, eq6], dtype=np.float64)

In [16]:
def Initial_Guess(x):
    
    a, b = Hexapod_Constructive_Vectors()
    
    P = ((x[0], x[1], x[2]))
    thetaRX, thetaRY, thetaRZ = (x[3], x[4], x[5])
    
    L1 = np.linalg.norm(P + Rotation_XYZ(thetaRX, thetaRY, thetaRZ).dot(b[0]) - a[0])
    L2 = np.linalg.norm(P + Rotation_XYZ(thetaRX, thetaRY, thetaRZ).dot(b[1]) - a[1])
    L3 = np.linalg.norm(P + Rotation_XYZ(thetaRX, thetaRY, thetaRZ).dot(b[2]) - a[2])
    L4 = np.linalg.norm(P + Rotation_XYZ(thetaRX, thetaRY, thetaRZ).dot(b[3]) - a[3])
    L5 = np.linalg.norm(P + Rotation_XYZ(thetaRX, thetaRY, thetaRZ).dot(b[4]) - a[4])
    L6 = np.linalg.norm(P + Rotation_XYZ(thetaRX, thetaRY, thetaRZ).dot(b[5]) - a[5])
    
    L = np.array([L1, L2, L3, L4, L5, L6], dtype=np.float64) # Should be actual encoder reading positions

    # Regression from Inverse Kinematics for the First Guess:
    Guess_X = ((-1.51694257e-9) + L1*(1.88740566) + L2*(-1.88740566) + L3*(-1.78630476e-1) + L4*(1.70265653) + L5*(-1.70265653) + L6*(1.7863047e-1));
    Guess_Y = ((-0.00958264) + L1*(-0.87999987) + L2*(-0.87999987) + L3*(2.07628491) + L4*(-1.19622617) + L5*(-1.19622617) + L6*(2.07628491));
    Guess_Z = ((-11.31748409) + L1*(0.17310112) + L2*(0.17310112) + L3*(0.1730323) + L4*(0.17318892) + L5*(0.17318892) + L6*(0.1730323));

    Guess_RX = ((-0.0012806) + L1*(0.09379136) + L2*(0.09379136) + L3*(0.60417547) + L4*(-0.69795643) + L5*(-0.69795643) + L6*(0.60417547));
    Guess_RY = ((6.14411133e-10) + L1*(-7.47210598e-1) + L2*(7.47210598e-1) + L3*(4.53910209e-1) + L4*(-2.92504249e-1) + L5*(2.92504249e-1) + L6*(-4.53910209e-1));
    Guess_RZ = ((-3.11851055e-10) + L1*(-1.22550406) + L2*(1.22550406) + L3*(-1.22207852) + L4*(1.22116327) + L5*(-1.22116327) + L6*(1.22207852));

    Init_Guess = np.array([Guess_X, Guess_Y, Guess_Z, Guess_RX, Guess_RY, Guess_RZ], dtype=np.float64)
    
    return Init_Guess, L

In [17]:
def Hexapod_Jacobian_System(X0):
    
    J = np.zeros((6,6), dtype=np.float64)

    J[0,0] = fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1-1.625252976396777E1)*(((X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1-1.625252976396777E1)/fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1-1.625252976396777E1)))*1.0/sqrt(pow(fabs(X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1),2.0)+pow(fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1-1.625252976396777E1),2.0)+pow(fabs(X0[2]-sin(X0[4]*1.74532925199433E-2)*2.099858217227977E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.500084252271299E1),2.0))*1.0;
    J[0,1] = fabs(X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1)*(((X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1)/fabs(X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1)))*1.0/sqrt(pow(fabs(X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1),2.0)+pow(fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1-1.625252976396777E1),2.0)+pow(fabs(X0[2]-sin(X0[4]*1.74532925199433E-2)*2.099858217227977E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.500084252271299E1),2.0))*1.0;
    J[0,2] = fabs(X0[2]-sin(X0[4]*1.74532925199433E-2)*2.099858217227977E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.500084252271299E1)*(((X0[2]-sin(X0[4]*1.74532925199433E-2)*2.099858217227977E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.500084252271299E1)/fabs(X0[2]-sin(X0[4]*1.74532925199433E-2)*2.099858217227977E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.500084252271299E1)))*1.0/sqrt(pow(fabs(X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1),2.0)+pow(fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1-1.625252976396777E1),2.0)+pow(fabs(X0[2]-sin(X0[4]*1.74532925199433E-2)*2.099858217227977E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.500084252271299E1),2.0))*1.0;
    J[0,3] = (fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1-1.625252976396777E1)*(((X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1-1.625252976396777E1)/fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1-1.625252976396777E1)))*(sin(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*7.854128681928129E-1+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*7.854128681928129E-1)*2.0-fabs(X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1)*(((X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1)/fabs(X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1)))*(cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*7.854128681928129E-1-cos(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*7.854128681928129E-1)*2.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[4]*1.74532925199433E-2)*fabs(X0[2]-sin(X0[4]*1.74532925199433E-2)*2.099858217227977E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.500084252271299E1)*(((X0[2]-sin(X0[4]*1.74532925199433E-2)*2.099858217227977E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.500084252271299E1)/fabs(X0[2]-sin(X0[4]*1.74532925199433E-2)*2.099858217227977E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.500084252271299E1)))*1.570825736385626)*1.0/sqrt(pow(fabs(X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1),2.0)+pow(fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1-1.625252976396777E1),2.0)+pow(fabs(X0[2]-sin(X0[4]*1.74532925199433E-2)*2.099858217227977E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.500084252271299E1),2.0))*5.0E-1;
    J[0,4] = (fabs(X0[2]-sin(X0[4]*1.74532925199433E-2)*2.099858217227977E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.500084252271299E1)*(((X0[2]-sin(X0[4]*1.74532925199433E-2)*2.099858217227977E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.500084252271299E1)/fabs(X0[2]-sin(X0[4]*1.74532925199433E-2)*2.099858217227977E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.500084252271299E1)))*(cos(X0[4]*1.74532925199433E-2)*3.664943971568651E-1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*7.854128681928129E-1)*2.0+fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1-1.625252976396777E1)*(((X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1-1.625252976396777E1)/fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1-1.625252976396777E1)))*(cos(X0[5]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*3.664943971568651E-1-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*7.854128681928129E-1)*2.0+fabs(X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1)*(((X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1)/fabs(X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1)))*(sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*3.664943971568651E-1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*7.854128681928129E-1)*2.0)*1.0/sqrt(pow(fabs(X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1),2.0)+pow(fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1-1.625252976396777E1),2.0)+pow(fabs(X0[2]-sin(X0[4]*1.74532925199433E-2)*2.099858217227977E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.500084252271299E1),2.0))*-5.0E-1;
    J[0,5] = (fabs(X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1)*(((X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1)/fabs(X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1)))*(cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*3.664943971568651E-1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*7.854128681928129E-1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*7.854128681928129E-1)*2.0-fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1-1.625252976396777E1)*(((X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1-1.625252976396777E1)/fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1-1.625252976396777E1)))*(cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*7.854128681928129E-1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*3.664943971568651E-1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*7.854128681928129E-1)*2.0)*1.0/sqrt(pow(fabs(X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1),2.0)+pow(fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1-1.625252976396777E1),2.0)+pow(fabs(X0[2]-sin(X0[4]*1.74532925199433E-2)*2.099858217227977E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.500084252271299E1),2.0))*5.0E-1;
    J[1,0] = fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1+1.625252976396777E1)*(((X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1+1.625252976396777E1)/fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1+1.625252976396777E1)))*1.0/sqrt(pow(fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1+1.625252976396777E1),2.0)+pow(fabs(X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1),2.0)+pow(fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*2.099858217227977E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.500084252271299E1),2.0))*1.0;
    J[1,1] = fabs(X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1)*(((X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1)/fabs(X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1)))*1.0/sqrt(pow(fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1+1.625252976396777E1),2.0)+pow(fabs(X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1),2.0)+pow(fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*2.099858217227977E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.500084252271299E1),2.0))*1.0;
    J[1,2] = fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*2.099858217227977E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.500084252271299E1)*(((X0[2]+sin(X0[4]*1.74532925199433E-2)*2.099858217227977E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.500084252271299E1)/fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*2.099858217227977E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.500084252271299E1)))*1.0/sqrt(pow(fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1+1.625252976396777E1),2.0)+pow(fabs(X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1),2.0)+pow(fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*2.099858217227977E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.500084252271299E1),2.0))*1.0;
    J[1,3] = 1.0/sqrt(pow(fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1+1.625252976396777E1),2.0)+pow(fabs(X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1),2.0)+pow(fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*2.099858217227977E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.500084252271299E1),2.0))*(fabs(X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1)*(((X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1)/fabs(X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1)))*(cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*7.854128681928129E-1-cos(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*7.854128681928129E-1)*-2.0+fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1+1.625252976396777E1)*(((X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1+1.625252976396777E1)/fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1+1.625252976396777E1)))*(sin(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*7.854128681928129E-1+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*7.854128681928129E-1)*2.0+fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*2.099858217227977E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.500084252271299E1)*(((X0[2]+sin(X0[4]*1.74532925199433E-2)*2.099858217227977E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.500084252271299E1)/fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*2.099858217227977E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.500084252271299E1)))*cos(X0[3]*1.74532925199433E-2)*cos(X0[4]*1.74532925199433E-2)*1.570825736385626)*5.0E-1;
    J[1,4] = (fabs(X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1)*(((X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1)/fabs(X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1)))*(sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*3.664943971568651E-1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*7.854128681928129E-1)*2.0+fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*2.099858217227977E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.500084252271299E1)*(((X0[2]+sin(X0[4]*1.74532925199433E-2)*2.099858217227977E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.500084252271299E1)/fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*2.099858217227977E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.500084252271299E1)))*(cos(X0[4]*1.74532925199433E-2)*3.664943971568651E-1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*7.854128681928129E-1)*2.0+fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1+1.625252976396777E1)*(((X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1+1.625252976396777E1)/fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1+1.625252976396777E1)))*(cos(X0[5]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*3.664943971568651E-1+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*7.854128681928129E-1)*2.0)*1.0/sqrt(pow(fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1+1.625252976396777E1),2.0)+pow(fabs(X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1),2.0)+pow(fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*2.099858217227977E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.500084252271299E1),2.0))*5.0E-1;
    J[1,5] = (fabs(X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1)*(((X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1)/fabs(X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1)))*(cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*3.664943971568651E-1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*7.854128681928129E-1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*7.854128681928129E-1)*2.0+fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1+1.625252976396777E1)*(((X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1+1.625252976396777E1)/fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1+1.625252976396777E1)))*(cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*7.854128681928129E-1-cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*3.664943971568651E-1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*7.854128681928129E-1)*2.0)*1.0/sqrt(pow(fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.099858217227977E1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.500084252271299E1+1.625252976396777E1),2.0)+pow(fabs(X0[1]+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.500084252271299E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.099858217227977E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.500084252271299E1-7.82397812897719E1),2.0)+pow(fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*2.099858217227977E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.500084252271299E1),2.0))*-5.0E-1;
    J[2,0] = fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208+7.588390306546476E1)*(((X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208+7.588390306546476E1)/fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208+7.588390306546476E1)))*1.0/sqrt(pow(fabs(X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-2.504478699352717E1),2.0)+pow(fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*4.947116390251234E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.315115656707208),2.0)+pow(fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208+7.588390306546476E1),2.0))*1.0;
    J[2,1] = fabs(X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-2.504478699352717E1)*(((X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-2.504478699352717E1)/fabs(X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-2.504478699352717E1)))*1.0/sqrt(pow(fabs(X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-2.504478699352717E1),2.0)+pow(fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*4.947116390251234E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.315115656707208),2.0)+pow(fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208+7.588390306546476E1),2.0))*-1.0;
    J[2,2] = fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*4.947116390251234E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.315115656707208)*(((X0[2]+sin(X0[4]*1.74532925199433E-2)*4.947116390251234E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.315115656707208)/fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*4.947116390251234E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.315115656707208)))*1.0/sqrt(pow(fabs(X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-2.504478699352717E1),2.0)+pow(fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*4.947116390251234E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.315115656707208),2.0)+pow(fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208+7.588390306546476E1),2.0))*1.0;
    J[2,3] = 1.0/sqrt(pow(fabs(X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-2.504478699352717E1),2.0)+pow(fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*4.947116390251234E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.315115656707208),2.0)+pow(fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208+7.588390306546476E1),2.0))*(fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208+7.588390306546476E1)*(((X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208+7.588390306546476E1)/fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208+7.588390306546476E1)))*(sin(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*7.531297581389812E-2+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*7.531297581389812E-2)*2.0+fabs(X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-2.504478699352717E1)*(((X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-2.504478699352717E1)/fabs(X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-2.504478699352717E1)))*(cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*7.531297581389812E-2-cos(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*7.531297581389812E-2)*2.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[4]*1.74532925199433E-2)*fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*4.947116390251234E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.315115656707208)*(((X0[2]+sin(X0[4]*1.74532925199433E-2)*4.947116390251234E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.315115656707208)/fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*4.947116390251234E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.315115656707208)))*1.506259516277962E-1)*-5.0E-1;
    J[2,4] = (fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*4.947116390251234E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.315115656707208)*(((X0[2]+sin(X0[4]*1.74532925199433E-2)*4.947116390251234E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.315115656707208)/fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*4.947116390251234E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.315115656707208)))*(cos(X0[4]*1.74532925199433E-2)*8.634346948926075E-1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*7.531297581389812E-2)*2.0-fabs(X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-2.504478699352717E1)*(((X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-2.504478699352717E1)/fabs(X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-2.504478699352717E1)))*(sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*8.634346948926075E-1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*7.531297581389812E-2)*2.0+fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208+7.588390306546476E1)*(((X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208+7.588390306546476E1)/fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208+7.588390306546476E1)))*(cos(X0[5]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*8.634346948926075E-1-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*7.531297581389812E-2)*2.0)*1.0/sqrt(pow(fabs(X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-2.504478699352717E1),2.0)+pow(fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*4.947116390251234E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.315115656707208),2.0)+pow(fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208+7.588390306546476E1),2.0))*5.0E-1;
    J[2,5] = (fabs(X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-2.504478699352717E1)*(((X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-2.504478699352717E1)/fabs(X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-2.504478699352717E1)))*(cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*8.634346948926075E-1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*7.531297581389812E-2+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*7.531297581389812E-2)*2.0+fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208+7.588390306546476E1)*(((X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208+7.588390306546476E1)/fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208+7.588390306546476E1)))*(cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*7.531297581389812E-2+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*8.634346948926075E-1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*7.531297581389812E-2)*2.0)*1.0/sqrt(pow(fabs(X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-2.504478699352717E1),2.0)+pow(fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*4.947116390251234E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.315115656707208),2.0)+pow(fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208+7.588390306546476E1),2.0))*5.0E-1;
    J[3,0] = (((X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023256E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600581E1+5.963137330149698E1)/fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023256E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600581E1+5.963137330149698E1)))*fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023256E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600581E1+5.963137330149698E1)*1.0/sqrt(pow(fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*2.847258173023256E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.068572686600581E1),2.0)+pow(fabs(X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600581E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023256E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-5.319499429624477E1),2.0)+pow(fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023256E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600581E1+5.963137330149698E1),2.0))*1.0;
    J[3,1] = fabs(X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600581E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023256E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-5.319499429624477E1)*(((X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600581E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023256E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-5.319499429624477E1)/fabs(X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600581E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023256E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-5.319499429624477E1)))*1.0/sqrt(pow(fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*2.847258173023256E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.068572686600581E1),2.0)+pow(fabs(X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600581E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023256E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-5.319499429624477E1),2.0)+pow(fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023256E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600581E1+5.963137330149698E1),2.0))*-1.0;
    J[3,2] = fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*2.847258173023256E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.068572686600581E1)*(((X0[2]+sin(X0[4]*1.74532925199433E-2)*2.847258173023256E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.068572686600581E1)/fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*2.847258173023256E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.068572686600581E1)))*1.0/sqrt(pow(fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*2.847258173023256E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.068572686600581E1),2.0)+pow(fabs(X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600581E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023256E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-5.319499429624477E1),2.0)+pow(fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023256E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600581E1+5.963137330149698E1),2.0))*1.0;
    J[3,3] = (fabs(X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600581E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023256E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-5.319499429624477E1)*(((X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600581E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023256E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-5.319499429624477E1)/fabs(X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600581E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023256E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-5.319499429624477E1)))*(cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*7.100998923789152E-1-cos(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*7.100998923789152E-1)*2.0+(((X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023256E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600581E1+5.963137330149698E1)/fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023256E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600581E1+5.963137330149698E1)))*fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023256E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600581E1+5.963137330149698E1)*(sin(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*7.100998923789152E-1+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*7.100998923789152E-1)*2.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[4]*1.74532925199433E-2)*fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*2.847258173023256E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.068572686600581E1)*(((X0[2]+sin(X0[4]*1.74532925199433E-2)*2.847258173023256E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.068572686600581E1)/fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*2.847258173023256E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.068572686600581E1)))*1.42019978475783)*1.0/sqrt(pow(fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*2.847258173023256E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.068572686600581E1),2.0)+pow(fabs(X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600581E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023256E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-5.319499429624477E1),2.0)+pow(fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023256E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600581E1+5.963137330149698E1),2.0))*-5.0E-1;
    J[3,4] = ((((X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023256E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600581E1+5.963137330149698E1)/fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023256E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600581E1+5.963137330149698E1)))*fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023256E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600581E1+5.963137330149698E1)*(cos(X0[5]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.969402977357421E-1-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*7.100998923789152E-1)*2.0+fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*2.847258173023256E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.068572686600581E1)*(((X0[2]+sin(X0[4]*1.74532925199433E-2)*2.847258173023256E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.068572686600581E1)/fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*2.847258173023256E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.068572686600581E1)))*(cos(X0[4]*1.74532925199433E-2)*4.969402977357421E-1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*7.100998923789152E-1)*2.0-fabs(X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600581E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023256E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-5.319499429624477E1)*(((X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600581E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023256E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-5.319499429624477E1)/fabs(X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600581E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023256E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-5.319499429624477E1)))*(sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.969402977357421E-1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*7.100998923789152E-1)*2.0)*1.0/sqrt(pow(fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*2.847258173023256E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.068572686600581E1),2.0)+pow(fabs(X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600581E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023256E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-5.319499429624477E1),2.0)+pow(fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023256E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600581E1+5.963137330149698E1),2.0))*5.0E-1;
    J[3,5] = (fabs(X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600581E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023256E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-5.319499429624477E1)*(((X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600581E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023256E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-5.319499429624477E1)/fabs(X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600581E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023256E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-5.319499429624477E1)))*(cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.969402977357421E-1-cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*7.100998923789152E-1+cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*7.100998923789152E-1)*2.0+(((X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023256E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600581E1+5.963137330149698E1)/fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023256E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600581E1+5.963137330149698E1)))*fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023256E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600581E1+5.963137330149698E1)*(cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*7.100998923789152E-1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.969402977357421E-1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*7.100998923789152E-1)*2.0)*1.0/sqrt(pow(fabs(X0[2]+sin(X0[4]*1.74532925199433E-2)*2.847258173023256E1-cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.068572686600581E1),2.0)+pow(fabs(X0[1]*-1.0+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600581E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023256E1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-5.319499429624477E1),2.0)+pow(fabs(X0[0]-cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023256E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600581E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600581E1+5.963137330149698E1),2.0))*5.0E-1;
    J[4,0] = fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600579E1-5.963137330149703E1)*(((X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600579E1-5.963137330149703E1)/fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600579E1-5.963137330149703E1)))*1.0/sqrt(pow(fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600579E1-5.963137330149703E1),2.0)+pow(fabs(X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600579E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023258E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1+5.31949942962447E1),2.0)+pow(fabs(X0[2]*-1.0+sin(X0[4]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.068572686600579E1),2.0))*1.0;
    J[4,1] = fabs(X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600579E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023258E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1+5.31949942962447E1)*(((X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600579E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023258E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1+5.31949942962447E1)/fabs(X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600579E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023258E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1+5.31949942962447E1)))*1.0/sqrt(pow(fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600579E1-5.963137330149703E1),2.0)+pow(fabs(X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600579E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023258E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1+5.31949942962447E1),2.0)+pow(fabs(X0[2]*-1.0+sin(X0[4]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.068572686600579E1),2.0))*1.0;
    J[4,2] = fabs(X0[2]*-1.0+sin(X0[4]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.068572686600579E1)*(((X0[2]*-1.0+sin(X0[4]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.068572686600579E1)/fabs(X0[2]*-1.0+sin(X0[4]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.068572686600579E1)))*1.0/sqrt(pow(fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600579E1-5.963137330149703E1),2.0)+pow(fabs(X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600579E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023258E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1+5.31949942962447E1),2.0)+pow(fabs(X0[2]*-1.0+sin(X0[4]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.068572686600579E1),2.0))*-1.0;
    J[4,3] = 1.0/sqrt(pow(fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600579E1-5.963137330149703E1),2.0)+pow(fabs(X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600579E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023258E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1+5.31949942962447E1),2.0)+pow(fabs(X0[2]*-1.0+sin(X0[4]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.068572686600579E1),2.0))*(fabs(X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600579E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023258E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1+5.31949942962447E1)*(((X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600579E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023258E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1+5.31949942962447E1)/fabs(X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600579E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023258E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1+5.31949942962447E1)))*(cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*7.100998923789149E-1-cos(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*7.100998923789149E-1)*2.0-fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600579E1-5.963137330149703E1)*(((X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600579E1-5.963137330149703E1)/fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600579E1-5.963137330149703E1)))*(sin(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*7.100998923789149E-1+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*7.100998923789149E-1)*2.0+fabs(X0[2]*-1.0+sin(X0[4]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.068572686600579E1)*cos(X0[3]*1.74532925199433E-2)*cos(X0[4]*1.74532925199433E-2)*(((X0[2]*-1.0+sin(X0[4]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.068572686600579E1)/fabs(X0[2]*-1.0+sin(X0[4]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.068572686600579E1)))*1.42019978475783)*5.0E-1;
    J[4,4] = 1.0/sqrt(pow(fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600579E1-5.963137330149703E1),2.0)+pow(fabs(X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600579E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023258E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1+5.31949942962447E1),2.0)+pow(fabs(X0[2]*-1.0+sin(X0[4]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.068572686600579E1),2.0))*(fabs(X0[2]*-1.0+sin(X0[4]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.068572686600579E1)*(((X0[2]*-1.0+sin(X0[4]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.068572686600579E1)/fabs(X0[2]*-1.0+sin(X0[4]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.068572686600579E1)))*(cos(X0[4]*1.74532925199433E-2)*4.969402977357424E-1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*7.100998923789149E-1)*-2.0+fabs(X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600579E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023258E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1+5.31949942962447E1)*(((X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600579E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023258E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1+5.31949942962447E1)/fabs(X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600579E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023258E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1+5.31949942962447E1)))*(sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.969402977357424E-1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*7.100998923789149E-1)*2.0+fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600579E1-5.963137330149703E1)*(((X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600579E1-5.963137330149703E1)/fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600579E1-5.963137330149703E1)))*(cos(X0[5]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.969402977357424E-1+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*7.100998923789149E-1)*2.0)*-5.0E-1;
    J[4,5] = (fabs(X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600579E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023258E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1+5.31949942962447E1)*(((X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600579E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023258E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1+5.31949942962447E1)/fabs(X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600579E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023258E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1+5.31949942962447E1)))*(cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.969402977357424E-1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*7.100998923789149E-1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*7.100998923789149E-1)*2.0+fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600579E1-5.963137330149703E1)*(((X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600579E1-5.963137330149703E1)/fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600579E1-5.963137330149703E1)))*(cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*7.100998923789149E-1-cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.969402977357424E-1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*7.100998923789149E-1)*2.0)*1.0/sqrt(pow(fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.068572686600579E1-5.963137330149703E1),2.0)+pow(fabs(X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.068572686600579E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*2.847258173023258E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.068572686600579E1+5.31949942962447E1),2.0)+pow(fabs(X0[2]*-1.0+sin(X0[4]*1.74532925199433E-2)*2.847258173023258E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.068572686600579E1),2.0))*5.0E-1;
    J[5,0] = fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208-7.588390306546476E1)*(((X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208-7.588390306546476E1)/fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208-7.588390306546476E1)))*1.0/sqrt(pow(fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208-7.588390306546476E1),2.0)+pow(fabs(X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208+2.504478699352713E1),2.0)+pow(fabs(X0[2]*-1.0+sin(X0[4]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.315115656707208),2.0))*1.0;
    J[5,1] = fabs(X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208+2.504478699352713E1)*(((X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208+2.504478699352713E1)/fabs(X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208+2.504478699352713E1)))*1.0/sqrt(pow(fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208-7.588390306546476E1),2.0)+pow(fabs(X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208+2.504478699352713E1),2.0)+pow(fabs(X0[2]*-1.0+sin(X0[4]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.315115656707208),2.0))*1.0;
    J[5,2] = fabs(X0[2]*-1.0+sin(X0[4]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.315115656707208)*(((X0[2]*-1.0+sin(X0[4]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.315115656707208)/fabs(X0[2]*-1.0+sin(X0[4]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.315115656707208)))*1.0/sqrt(pow(fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208-7.588390306546476E1),2.0)+pow(fabs(X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208+2.504478699352713E1),2.0)+pow(fabs(X0[2]*-1.0+sin(X0[4]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.315115656707208),2.0))*-1.0;
    J[5,3] = 1.0/sqrt(pow(fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208-7.588390306546476E1),2.0)+pow(fabs(X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208+2.504478699352713E1),2.0)+pow(fabs(X0[2]*-1.0+sin(X0[4]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.315115656707208),2.0))*(fabs(X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208+2.504478699352713E1)*(((X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208+2.504478699352713E1)/fabs(X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208+2.504478699352713E1)))*(cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*7.531297581389812E-2-cos(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*7.531297581389812E-2)*2.0-fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208-7.588390306546476E1)*(((X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208-7.588390306546476E1)/fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208-7.588390306546476E1)))*(sin(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*7.531297581389812E-2+cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*7.531297581389812E-2)*2.0+fabs(X0[2]*-1.0+sin(X0[4]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.315115656707208)*(((X0[2]*-1.0+sin(X0[4]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.315115656707208)/fabs(X0[2]*-1.0+sin(X0[4]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.315115656707208)))*cos(X0[3]*1.74532925199433E-2)*cos(X0[4]*1.74532925199433E-2)*1.506259516277962E-1)*5.0E-1;
    J[5,4] = 1.0/sqrt(pow(fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208-7.588390306546476E1),2.0)+pow(fabs(X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208+2.504478699352713E1),2.0)+pow(fabs(X0[2]*-1.0+sin(X0[4]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.315115656707208),2.0))*(fabs(X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208+2.504478699352713E1)*(((X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208+2.504478699352713E1)/fabs(X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208+2.504478699352713E1)))*(sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*8.634346948926075E-1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*7.531297581389812E-2)*2.0+fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208-7.588390306546476E1)*(((X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208-7.588390306546476E1)/fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208-7.588390306546476E1)))*(cos(X0[5]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*8.634346948926075E-1+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*7.531297581389812E-2)*2.0-fabs(X0[2]*-1.0+sin(X0[4]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.315115656707208)*(((X0[2]*-1.0+sin(X0[4]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.315115656707208)/fabs(X0[2]*-1.0+sin(X0[4]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.315115656707208)))*(cos(X0[4]*1.74532925199433E-2)*8.634346948926075E-1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*7.531297581389812E-2)*2.0)*-5.0E-1;
    J[5,5] = (fabs(X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208+2.504478699352713E1)*(((X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208+2.504478699352713E1)/fabs(X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208+2.504478699352713E1)))*(cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*8.634346948926075E-1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*7.531297581389812E-2-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*7.531297581389812E-2)*2.0+fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208-7.588390306546476E1)*(((X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208-7.588390306546476E1)/fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208-7.588390306546476E1)))*(cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*7.531297581389812E-2-cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*8.634346948926075E-1+sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*7.531297581389812E-2)*2.0)*1.0/sqrt(pow(fabs(X0[0]+cos(X0[4]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[3]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208-cos(X0[5]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*4.315115656707208-7.588390306546476E1),2.0)+pow(fabs(X0[1]-cos(X0[3]*1.74532925199433E-2)*cos(X0[5]*1.74532925199433E-2)*4.315115656707208+cos(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.947116390251234E1-sin(X0[3]*1.74532925199433E-2)*sin(X0[4]*1.74532925199433E-2)*sin(X0[5]*1.74532925199433E-2)*4.315115656707208+2.504478699352713E1),2.0)+pow(fabs(X0[2]*-1.0+sin(X0[4]*1.74532925199433E-2)*4.947116390251234E1+cos(X0[4]*1.74532925199433E-2)*sin(X0[3]*1.74532925199433E-2)*4.315115656707208),2.0))*5.0E-1;

    return J

In [18]:
def Newton(x, L, tol):
    
    start = time.time()
    
    F_value = Hexapod_System(x, L) # Define function with input L and guess vector x
    F_error = np.max(np.absolute(F_value))
    iter_counter = 0
    
    while ((F_error > tol) & (iter_counter < 100)):
        
        delta = np.linalg.solve(Hexapod_Jacobian_System(x), -F_value)
        x = x + delta
        F_value = Hexapod_System(x, L)
        F_error = np.max(np.absolute(F_value))
        iter_counter += 1
        
    end = time.time()
    total_time = end-start
    
    return x, iter_counter, total_time

In [19]:
def Newton_Test(True_Values, tol, print_process):
    
    Init_Guess, L = Initial_Guess(True_Values)

    Answer, n, total_time = Newton(Init_Guess, L, tol)
    
    max_error = np.max(np.absolute(True_Values - Answer))
    
    if (print_process==1):
        print('\n')
        print('Answer:        ', np.array(Answer, dtype=np.float32), '  N° of Iterations: ', n, '  Total Time: ', '{:.3f} ms'.format(1000*total_time))
        print('True:          ', True_Values)
        print('Max Error: ', max_error)
        print('Initial Guess: ', Init_Guess)
        print('###############################################################################################################')
    
    return max_error, n, total_time

In [652]:
True_Values = np.array([10.254, -8.254, 90.254, 1.254, 0.254, -5.254], dtype=np.float64)

Newton_Test(True_Values, tol=1e-8, print_process=1)



Answer:         [10.254 -8.254 90.254  1.254  0.254 -5.254]   N° of Iterations:  3   Total Time:  35.905 ms
True:           [10.254 -8.254 90.254  1.254  0.254 -5.254]
Max Error:  9.521272659185342e-13
Initial Guess:  [12.02342585 -9.78421509 89.72712996  1.66029009 -0.2149323  -6.1974919 ]
###############################################################################################################


(9.521272659185342e-13, 3, 0.03590512275695801)

In [20]:
def Test_Newton_X_times(N_times, tol=1e-8):

    error = np.zeros((N_times,), dtype = np.float64)
    n_iters = np.zeros((N_times,), dtype = np.float64)
    times = np.zeros((N_times,), dtype = np.float64)

    for i in range(N_times):
        x = random.uniform(-20.0, 20.0)
        y = random.uniform(-20.0, 20.0)
        z = random.uniform(80.0, 120.0)

        rx = random.uniform(-20.0, 20.0)
        ry = random.uniform(-20.0, 20.0)
        rz = random.uniform(-20.0, 20.0)

        True_Values = np.array([x, y, z, rx, ry, rz], dtype=np.float64)
        error[i], n_iters[i], times[i] = Newton_Test(True_Values, tol, print_process=0)
        clear_output(wait=True)
        print(i+1,'/',N_times)
        
    print('Max Error among the tests: ', np.max(error))
    print('Max Number of Iterations among the tests: ', np.max(n_iters))
    print('Min Number of Iterations among the tests: ', np.min(n_iters))
    print('Longest Runtime: ', '{:.5f} ms'.format(np.max(times)))
    print('Fastest Runtime: ', '{:.5f} ms'.format(np.min(times)))

In [658]:
Test_Newton_X_times(1000000, tol=1e-10)

1000000 / 1000000
Max Error among the tests:  2.7557334192351846e-10
Max Number of Iterations among the tests:  5.0
Min Number of Iterations among the tests:  3.0
Longest Runtime:  0.32014 ms
Fastest Runtime:  0.00202 ms


In [29]:
Test_Newton_X_times(50, tol=1e-10)

50 / 50
Max Error among the tests:  5.596234586846549e-11
Max Number of Iterations among the tests:  4.0
Min Number of Iterations among the tests:  3.0
Longest Runtime:  0.04488 ms
Fastest Runtime:  0.02094 ms
